# Visualize and analyze data

## Package imports

In [ ]:
import sys
import os
import urllib.request

# installing needed packages - this may take a few minutes
try:
    import shapely
    import shapely.geometry
    import folium
except ModuleNotFoundError:
    !{sys.executable} -m pip install shapely
    !{sys.executable} -m pip install folium
    import shapely
    import shapely.geometry
    import folium


try:
    import matplotlib.pyplot as plt
    from mpl_toolkits import mplot3d
except ModuleNotFoundError:
    !{sys.executable} -m pip install matplotlib
    import matplotlib.pyplot as plt
    from mpl_toolkits import mplot3d

try:
    import laspy
except ModuleNotFoundError:
    !{sys.executable} -m pip install laspy[laszip]
    import laspy

import numpy as np
from pathlib import Path
import tempfile

sys.path.append("..")
from pytreedb import db

## Import database

Define (local) MongoDB connection and import database from URL

In [ ]:
mydbfile = "my_pytree.db"
db_url = "https://github.com/3dgeo-heidelberg/pytreedb/raw/main/data/test/data.db"
mydb = db.PyTreeDB(dbfile=mydbfile)
# mydb.import_db(db_url, overwrite=True)

## Show data on a map

First, let's query the database and select only trees for which terrestrial laser scanning (TLS) data is available. 

In [ ]:
res = mydb.query({"properties.data.mode": "TLS"})

To display the filtered trees on the map, we retrieve their locations, their IDs and their species.

In [ ]:
tree_locations = [tree["geometry"]["coordinates"] for tree in res]
tree_ids = [tree["properties"]["id"] for tree in res]
tree_species = [tree["properties"]["species"] for tree in res]

You can zoom in and click on the markers in the map created below to read out the ID and species of the respective tree.

In [ ]:
tooltip = "Click me!"
m = folium.Map([49.0077, 8.702], zoom_start=13)
for point, name, species in zip(tree_locations, tree_ids, tree_species):
    html = f"""
    <strong>{name}</strong><br>
    <em>{species}</em>
    """
    # note that we have to reverse latitude and longitude
    folium.Marker(point[1::-1], popup=html, tooltip=tooltip).add_to(m)
m

In [ ]:
tooltip = "Click me!"
m = folium.Map([49.0299, 8.4382], zoom_start=14)
for point, name, species in zip(tree_locations, tree_ids, tree_species):
    html = f"""
    <strong>{name}</strong><br>
    <em>{species}</em>
    """
    # note that we have to reverse latitude and longitude
    folium.Marker(point[1::-1], popup=html, tooltip=tooltip).add_to(m)
m

## Bar plots and Histograms

Plot the number of available trees for each data source as a bar plot

In [ ]:
data = {
    "ALS": len(mydb.query({"properties.data.mode": "ALS"})),
    # "$elemMatch" operator to specify multiple criteria on an array element.
    "ULS leaf-on": len(mydb.query({"properties.data": {"$elemMatch": {"mode": "ULS", "canopy_condition": "leaf-on"}}})),
    "ULS leaf-off": len(
        mydb.query({"properties.data": {"$elemMatch": {"mode": "ULS", "canopy_condition": "leaf-off"}}})
    ),
    "TLS": len(mydb.query({"properties.data.mode": "TLS"})),
}

In [ ]:
names = list(data.keys())
values = list(data.values())

plots = plt.bar(range(len(data)), values, tick_label=names)
for bar in plots.patches:
    plt.annotate(
        format(bar.get_height(), "d"),
        (bar.get_x() + bar.get_width() / 2, bar.get_height()),
        ha="center",
        va="center",
        size=13,
        xytext=(0, -10),
        textcoords="offset points",
    )
plt.show()

Plot a histogram of tree heights (measured from ALS data)

In [ ]:
ALS_meas = [
    meas_dict
    for tree in mydb.db
    for meas_dict in tree["properties"]["measurements"]
    if "source" in meas_dict.keys() and meas_dict["source"] == "ALS"
]
tree_heights_ALS = np.array([meas_dict["height_m"] for meas_dict in ALS_meas])

# the histogram of the data
n, bins, patches = plt.hist(tree_heights_ALS, 50, facecolor="g", alpha=0.75)

plt.xlabel("Tree height [m]")
plt.ylabel("Frequency")
plt.title("Histogram of tree heights")
plt.grid(True)
plt.show()

## Scatter plot of metrics

Scatter plots of metrics obtained from different sources

In [ ]:
res = mydb.query({"$and": [{"properties.data.mode": "ALS"}, {"properties.data.mode": "ULS"}]})
ULS_meas = [
    meas_dict
    for tree in res
    for meas_dict in tree["properties"]["measurements"]
    if "source" in meas_dict.keys() and meas_dict["source"] == "ULS" and meas_dict["canopy_condition"] == "leaf-on"
]
ALS_meas = [
    meas_dict
    for tree in res
    for meas_dict in tree["properties"]["measurements"]
    if "source" in meas_dict.keys() and meas_dict["source"] == "ALS" and meas_dict["canopy_condition"] == "leaf-on"
]
tree_heights_ULS = np.array([meas_dict["height_m"] for meas_dict in ULS_meas])
tree_heights_ALS = np.array([meas_dict["height_m"] for meas_dict in ALS_meas])

fig, ax = plt.subplots(figsize=(5, 5))
# scatter plot of tree heights from different sources
ax.scatter(tree_heights_ALS, tree_heights_ULS, s=8.0, alpha=0.2, color="green")
# plot diagonal
ax.plot([0, 1], [0, 1], transform=ax.transAxes, color="black", ls="--", lw=1.5)
plt.xlabel("source: ALS")
plt.ylabel("source: ULS")
plt.title("Tree height [m] estimated from ALS and ULS point clouds")

plt.axis("square")
plt.show()

In [ ]:
res = mydb.query({"$and": [{"properties.measurements.source": "TLS"}, {"properties.measurements.source": "FI"}]})
TLS_meas = [
    meas_dict
    for tree in res
    for meas_dict in tree["properties"]["measurements"]
    if "source" in meas_dict.keys() and meas_dict["source"] == "TLS"
]
FI_meas = [
    meas_dict
    for tree in res
    for meas_dict in tree["properties"]["measurements"]
    if "source" in meas_dict.keys() and meas_dict["source"] == "FI"
]
DBH_TLS = np.array([meas_dict["DBH_cm"] for meas_dict in TLS_meas])
DBH_FI = np.array([meas_dict["DBH_cm"] for meas_dict in FI_meas])

fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(DBH_TLS, DBH_FI, s=10, alpha=1.0, color="skyblue")
ax.plot([0, 1], [0, 1], transform=ax.transAxes, color="black", ls="--", lw=1.5)
plt.xlabel("source: TLS")
plt.ylabel("source: FI")
plt.title("DBH [cm] estimated from TLS point clouds and collected in the Forest Inventory")

plt.axis("square")
plt.show()

Plot DBH against tree height

In [ ]:
# "$elemMatch" operator to specify multiple criteria on an array element.
# the query returns trees which have both a) TLS measurements and b) leaf-on ULS measurements
res = mydb.query(
    {
        "$and": [
            {"properties.measurements.source": "TLS"},
            {"properties.measurements": {"$elemMatch": {"source": "ULS", "canopy_condition": "leaf-on"}}},
        ]
    }
)
TLS_meas = [
    meas_dict
    for tree in res
    for meas_dict in tree["properties"]["measurements"]
    if "source" in meas_dict.keys() and meas_dict["source"] == "TLS"
]
ULS_meas = [
    meas_dict
    for tree in res
    for meas_dict in tree["properties"]["measurements"]
    if "source" in meas_dict.keys() and meas_dict["source"] == "ULS" and meas_dict["canopy_condition"] == "leaf-on"
]

dbh = np.array([meas_dict["DBH_cm"] for meas_dict in TLS_meas])
height = np.array([meas_dict["height_m"] for meas_dict in ULS_meas])

fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(dbh, height, s=10, alpha=1.0, color="green")
plt.xlabel("DBH [cm]")
plt.ylabel("Tree height [m]")
plt.title("DBH against tree height")

plt.axis("square")
plt.show()

Compare crown diameter to tree height ratio for different species

In [ ]:
res = mydb.query(
    {
        "properties.measurements": {
            "$elemMatch": {
                "source": "ULS",
                "canopy_condition": "leaf-on",
                "height_m": {"$exists": True},
                "mean_crown_diameter_m": {"$exists": True},
            }
        }
    }
)
tree_species = [tree["properties"]["species"] for tree in res]
height = [
    meas_dict["height_m"]
    for tree in res
    for meas_dict in tree["properties"]["measurements"]
    if "source" in meas_dict.keys() and meas_dict["source"] == "ULS" and meas_dict["canopy_condition"] == "leaf-on"
]
diam = [
    meas_dict["mean_crown_diameter_m"]
    for tree in res
    for meas_dict in tree["properties"]["measurements"]
    if "source" in meas_dict.keys() and meas_dict["source"] == "ULS" and meas_dict["canopy_condition"] == "leaf-on"
]

num_colors = len(set(tree_species))
cm = plt.get_cmap("gist_rainbow")
my_colors = [cm(1.0 * i / num_colors) for i in range(num_colors)]

my_color_dict = {}
for i, species in enumerate(set(tree_species)):
    my_color_dict[species] = my_colors[i]

scatter = plt.scatter(diam, height, c=[my_color_dict[i] for i in tree_species])
plt.xlabel("Crown diameter [m]")
plt.ylabel("Tree height [m]")
plt.title("Crown diameter and tree height for trees by species")
plt.grid(True)
# generate custom fake lines that will be used as legend entries
markers = [plt.Line2D([0, 0], [0, 0], color=color, marker="o", linestyle="") for color in my_color_dict.values()]
plt.legend(markers, my_color_dict.keys(), numpoints=1, title="Species", loc="center left", bbox_to_anchor=(1, 0.5))
plt.show()

## 3D plots of point clouds

In [ ]:
# pypotree / PPTK / Open3D / PyntCloud / matplotlib ?
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

In [ ]:
res = mydb.query(
    {
        "$and": [
            {"properties.data": {"$elemMatch": {"mode": "TLS", "quality": 1}}},
            {"properties.data.mode": "ALS"},
            {"properties.data": {"$elemMatch": {"mode": "ULS", "canopy_condition": "leaf-on"}}},
            {"properties.data": {"$elemMatch": {"mode": "ULS", "canopy_condition": "leaf-off"}}},
            {"properties.species": "Quercus petraea"},
        ]
    }
)

In [ ]:
def reporthook(count, block_size, total_size):
    percent = min(int(count * block_size * 100 / total_size), 100)
    print("\r...{}%".format(percent), end="")


def download_data(filename, url):
    if not Path(filename).exists():
        print(f"Downloading data from '{url}' to '{filename}'. Please wait ...")
        if not Path(filename).parent.exists():
            Path(filename).parent.mkdir()
        urllib.request.urlretrieve(url, filename, reporthook=reporthook)
        print("\nDownload finished")
    else:
        print("File already exists. Great!")
    return

In [ ]:
output_folder = tempfile.TemporaryDirectory()
download_links = mydb.get_pointcloud_urls([res[0]])
modes = [data["mode"] for data in res[0]["properties"]["data"]]
canopy_cond = [data["canopy_condition"] for data in res[0]["properties"]["data"]]
files_written = []
for url in download_links:
    outfile = Path(output_folder.name) / url.split("/")[-1]
    download_data(outfile, url)
    files_written.append(outfile)

In [ ]:
def set_axes_equal(ax):
    """Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc..  This is one possible solution to Matplotlib's
    ax.set_aspect('equal') and ax.axis('equal') not working for 3D.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    """

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    # The plot bounding box is a sphere in the sense of the infinity
    # norm, hence I call half the max range the plot radius.
    plot_radius = 0.5 * max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])

In [ ]:
fig = plt.figure(figsize=(10, 50))

for i, file in enumerate(files_written):
    mode = modes[i]
    canopy = canopy_cond[i]
    ax = fig.add_subplot(5, 1, i + 1, projection="3d")
    las = laspy.read(file)
    coords = np.vstack((las.x, las.y, las.z)).transpose()
    try:
        reflectance = las.Reflectance
    except AttributeError:
        reflectance = las._Reflectance
    if mode == "TLS-on":
        coords = coords[::10]
        reflectance = reflectance[::10]
    ax.scatter(coords[:, 0], coords[:, 1], coords[:, 2], c=reflectance, s=0.01)
    ax.set_title(f"{mode} ({canopy})")
    set_axes_equal(ax)
    plt.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

plt.show()

In [ ]:
output_folder.cleanup()